#  Setup stuff: Connectivity

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 8Q6mTZrgrjzL63x1UzPSyNQw8KQzGzVMqDjhKiW7M6F3, 1>


#  Finally now, the Analytics ..

In [6]:

#  What's in the graph ..

l_result1 = my_graph.query_unpaginated("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query_unpaginated("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------+----------|
|  0 |               0 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node     |
|  1 | 281474976710656 | ['Airport'] | Airport   | SJC              | San Jose         | node     |
|  2 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        | node     |
|  3 | 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           | node     |
+----+-----------------+-------------+-----------+------------------+------------------+----------+


/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
|  0 | 562949953421313 | ['Airport']    |               0 | FLIES_TO  |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  1 | 562949953421312 | ['Airport']    |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  2 |               0 | ['Airport']    | 281474976710656 | FLIES_TO  |         1829 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
|  3 | 562949953421313 | ['Airport

In [ ]:

#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN ID(n) AS FROM, ID(m) AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS, TYPE(r) AS my_type, LABELS(n) AS FROM_LABELS, LABELS(m) AS TO_LABELS
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

l_result = my_graph.query("""

   MATCH (n: Airport) - [ r ] -> (m: Airport)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:
#################################################################

In [ ]:

#  https://stackoverflow.com/questions/27600608/traverse-path-cypher-query
#
#  Just direct flight between SJC and MKE   (none, there is no direct flight)

l_result = my_graph.query("""

   MATCH (n: Airport ) - [ r ] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:

#  First traversal, Airport --> Airport, up to 3 in distance ..

l_result = my_graph.query("""

   MATCH (n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code, m.airport_code, "3" AS my_length
   
   """)

print(l_result[0:30])

#  Sample output,
#
#       n.airport_code m.airport_code my_length
#     0            SJC            MKE         3
#     1            SJC            MKE         3


In [8]:

#  Adding path ..

l_result = my_graph.query_unpaginated("""

   MATCH p= ( n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, m.airport_code AS TO, p
   """)

print(l_result[0:30])

#  Currently, error ..
#
#  OperationError: query 9tWneccAey6jzpKLGGtto7v3X1iSkQDdRrSa7gcuv8fD-3qJysJrdkEo1NWz7V error: backtrace
#  (QueryClient.cpp:456): opgraph check failed (OpGraph.cpp:429): Unimplemented Features Pass
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:360): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:658): backtrace (UnimplementedFeaturesChecker.cpp:214):
#  accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:24): InternalError:NotImplemented


          0/? [?op/s]

OperationError: 67Zur3wA57WX4BFcAKjAoyMe1mUc4wxQMfQmotcugkQU-5ehh6bsVPLK12jR7f backtrace (QueryOperation.cpp:66): backtrace (Network.h:331): backtrace (QueryOperation.cpp:69): backtrace (QueryClient.cpp:655): opgraph check failed (OpGraph.cpp:494): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:386): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:729): backtrace (UnimplementedFeaturesChecker.cpp:230): accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:23): InternalError:NotImplemented: InternalError:NotImplemented: InternalError:NotImplemented

In [9]:

#  Using COLLECT to try and get path ..

l_result = my_graph.query_unpaginated("""

   MATCH ( n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   
   RETURN n.airport_code AS FROM, COLLECT(m.airport_code)
   // RETURN COLLECT( [n.airport_code, m.airport_code] )                //  This did nothing effective
   
   """)

print(l_result[0:30])

#  This did not used to error. It used to return,
#
#    COLLECT(m.airport_code) FROM
#  0              [MKE, MKE]  SJC
# 
#  Not what we want

#  Now this gives,
#
#  OperationError: 9HaUVQpn6SQh83sR3oDGZLmtyzUoBgxEZeU26316cb4B-3Hc5zx68K6VVZyswB backtrace
#  (QueryClient.cpp:575): opgraph check failed (OpGraph.cpp:564): Unimplemented Features Pass 
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace 
#  (OpGraph.cpp:822): backtrace (OpGraph.cpp:822): backtrace (OpGraph.cpp:798):
#  returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197):
#  InternalError:NotImplemented: InternalError:NotImplemented


          0/? [?op/s]

OperationError: BWGoW7Yjer7TSGZfKBAHNjvcsQc7Xh5uUwbKMhm3d7hr-2RDBMc4j8Hh6n3Zin backtrace (QueryOperation.cpp:66): backtrace (Network.h:331): backtrace (QueryOperation.cpp:69): backtrace (QueryClient.cpp:655): opgraph check failed (OpGraph.cpp:494): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:386): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:729): returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:186): InternalError:NotImplemented: InternalError:NotImplemented: InternalError:NotImplemented

In [ ]:

#  Another attempt at using COLLECT, on the edge this time ..

l_result = my_graph.query("""

   MATCH ( n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, COLLECT(r.END_ID)
   """)

print(l_result[0:30])


#  OperationError: query 6jxiUgnCFao3yf2BTXPJqQDUTkAwTL4LMzmKjF6kNmD4-4BeDpVqisXe9Ha8QW error:
#  backtrace (QueryClient.cpp:452): backtrace (OpGraph.cpp:313): backtrace (OpGraph.h:588):
#  backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace
#  (OpGraph.cpp:320): backtrace (RewritePattern.h:46): backtrace (MatchOrderOptimizer.cpp:102):
#  backtrace (MatchOperator.cpp:2062): returning named variable paths not yet supported
#  (MatchOperator.cpp:1894): InternalError:NotImplemented


In [ ]:
# ##################################################################

In [17]:

#  Second traversal, Airport --> Airport, up to 3 in distance, but in other direction ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "MKE" AND m.airport_code = "SJC"
   RETURN n.airport_code, m.airport_code
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+------------------+------------------+
|    | n.airport_code   | m.airport_code   |
|----+------------------+------------------|
|  0 | MKE              | SJC              |
+----+------------------+------------------+


In [ ]:
# ##################################################################

In [19]:

#  From Roshan and Boris ..

l_result = my_graph.query("""

   MATCH (n: Airport ) - [] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN "1" AS num_hops, n.airport_code AS FROM, [m.airport_code] as path
      
   UNION
   
   MATCH (n: Airport) - [] -> (p1: Airport) - [] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN "2" AS num_hops, n.airport_code AS FROM, [p1.airport_code, m.airport_code] as path
      
   UNION
   
   MATCH (n: Airport) - [] -> (p1: Airport) - [] -> (p2: Airport) - [] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN "3" AS num_hops, n.airport_code AS FROM, [p1.airport_code, p2.airport_code, m.airport_code] as path
   
   """)

print(l_result[0:30])
    
#  Outputs,
#
#    FROM num_hops             path
#  0  SJC        3  [DEN, ORD, MKE]
#  1  SJC        2       [ORD, MKE]



          0/? [?op/s]

              path FROM num_hops
0  [DEN, ORD, MKE]  SJC        3
1       [ORD, MKE]  SJC        2
